In [32]:
import wrds
import pandas as pd

db = wrds.Connection(wrds_username="priyamvadadaga")

Loading library list...
Done


In [33]:
db.describe_table('comp', 'names')

Approximately 44198 rows in comp.names.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),Global Company Key
1,conm,True,VARCHAR(70),Company Name
2,tic,True,VARCHAR(8),Ticker Symbol
3,cusip,True,VARCHAR(20),CUSIP
4,cik,True,VARCHAR(10),CIK Number
5,sic,True,VARCHAR(4),Standard Industry Classification Code
6,naics,True,VARCHAR(6),North American Industry Classification Code
7,gsubind,True,VARCHAR(8),GIC Sub-Industries
8,gind,True,VARCHAR(6),GIC Industries
9,year1,True,DOUBLE PRECISION,First Fiscal Year of Available Accounting Data


In [34]:
names_query = """
    SELECT DISTINCT gvkey, tic AS ticker, conm AS company_name, gind, gsubind, sic, naics
    FROM comp.names
    WHERE tic IS NOT NULL
"""

names_df = db.raw_sql(names_query)

# filter: GICS sector 45 (Information Technology)
names_with_gind = names_df.dropna(subset=["gind"]).copy()
names_with_gind.loc[:,"gind_str"] = names_with_gind["gind"].astype(str)

tech_universe = names_with_gind[
    names_with_gind["gind_str"].str.startswith("45")
].drop(columns=["gind_str"]).drop_duplicates("gvkey")

In [35]:
mktcap_query = """
    WITH latest AS (
        SELECT gvkey, datadate, mkvalt, prcc_f, csho,
            ROW_NUMBER() OVER (
                PARTITION BY gvkey
                ORDER BY datadate DESC
            ) AS rn
        FROM comp.funda
        WHERE indfmt = 'INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C'
    )
    SELECT gvkey, datadate, mkvalt, prcc_f, csho
    FROM latest
    WHERE rn = 1
"""

mktcap_df = db.raw_sql(mktcap_query)

In [36]:
import numpy as np

mktcap_df.loc[:,"market_cap"] = mktcap_df["mkvalt"]
missing_mask = mktcap_df["market_cap"].isna()
mktcap_df.loc[missing_mask, "market_cap"] = (
    mktcap_df.loc[missing_mask, "prcc_f"] * mktcap_df.loc[missing_mask, "csho"]
)

mktcap_df = mktcap_df.dropna(subset=["market_cap"])

In [37]:
tech_with_mktcap = tech_universe.merge(mktcap_df[["gvkey", "market_cap"]], on="gvkey", how="left")
tech_with_mktcap = tech_with_mktcap.dropna(subset=["market_cap"])

tech_sorted = tech_with_mktcap.sort_values("market_cap", ascending=False)
print("Tech names with size info:", len(tech_sorted))

Tech names with size info: 4503


In [38]:
top_tech = tech_sorted.head(20)

In [39]:
top_tech

,gvkey,ticker,company_name,gind,gsubind,sic,naics,market_cap
1358,001690,AAPL,APPLE INC,452020,45202030,3663,334220,3761715.1938
3865,012141,MSFT,MICROSOFT CORP,451030,45103020,7372,513210,3697745.94
4699,117768,NVDA,NVIDIA CORP,453010,45301020,3674,334413,2938953.39
1189,201395,TSM,TAIWAN SEMICONDUCTOR MFG CO,453010,45301020,3674,334413,1024291.16703
799,180711,AVGO,BROADCOM INC,453010,45301020,3674,334413,795542.22
4640,012142,ORCL,ORACLE CORP,451030,45103020,7372,513210,464642.71
2626,157855,CRM,SALESFORCE INC,451030,45103010,7372,513210,328715.4
3141,103487,SAP,SAP SE,451030,45103010,7372,513210,287225.87769
3780,061214,ASML,ASML HOLDING NV,453010,45301010,3559,333242,272577.27472
2955,020779,CSCO,CISCO SYSTEMS INC,452010,45201020,3576,334118,269596.8


In [40]:
import os

UNIVERSE_PATH = "data/universe"
os.makedirs(UNIVERSE_PATH, exist_ok=True)

TOP_TECH_PARQUET = os.path.join(UNIVERSE_PATH, "tech_universe_top20.parquet")

top_tech.to_parquet(TOP_TECH_PARQUET, index=False)
print(f"Saved tech universe to {TOP_TECH_PARQUET}")

Saved tech universe to data/universe/tech_universe_top20.parquet


In [41]:
top_tech = pd.read_parquet("data/universe/tech_universe_top20.parquet")

link_query = """
    SELECT DISTINCT companyid, gvkey, ticker, companyname
    FROM ciq.wrds_ciqsymbol_primary
    WHERE gvkey IS NOT NULL AND companyid IS NOT NULL
"""
ciq_links = db.raw_sql(link_query)

ciq_links.loc[:, "gvkey"] = ciq_links["gvkey"].astype(str)
top_tech.loc[:, "gvkey"] = top_tech["gvkey"].astype(str)

top_tech_with_ciq = top_tech.merge(
    ciq_links.rename(columns={
        "companyid": "ciq_company_id",
        "companyname": "ciq_company_name"
    }),
    on="gvkey",
    how="inner"
)

top_tech_with_ciq = top_tech_with_ciq.dropna(subset=["ciq_company_id"])
top_tech_with_ciq = top_tech_with_ciq.drop_duplicates("ciq_company_id")

print("Top tech with CIQ IDs:", len(top_tech_with_ciq))

Top tech with CIQ IDs: 20


In [42]:
TECH_CIQ_PARQUET = os.path.join(UNIVERSE_PATH, "tech_universe_top20_with_ciq.parquet")
top_tech_with_ciq.to_parquet(TECH_CIQ_PARQUET, index=False)
print("Saved:", TECH_CIQ_PARQUET)

Saved: data/universe/tech_universe_top20_with_ciq.parquet


In [43]:
top_tech_with_ciq = pd.read_parquet(TECH_CIQ_PARQUET)
top_tech_with_ciq.loc[:, "ciq_company_id"] = top_tech_with_ciq["ciq_company_id"].astype(int)
company_ids = top_tech_with_ciq["ciq_company_id"].tolist()

In [53]:
ids = ", ".join(str(cid) for cid in company_ids)

sql_query = f"""
SELECT 
    d.companyid, d.transcriptid, d.headline,
    d.mostimportantdateutc, d.mostimportanttimeutc,
    d.keydeveventtypeid, d.keydeveventtypename,
    d.companyname AS detail_companyname,
    d.transcriptcollectiontypeid, p.transcriptcomponenttypeid,
    p.transcriptcomponenttypename, p.transcriptpersonid,
    p.transcriptpersonname, p.proid, p.companyofperson,
    p.speakertypeid, p.speakertypename,
    p.componentorder,
    p.componenttextpreview,
    c.componenttext
FROM ciq.wrds_transcript_detail d
JOIN ciq.wrds_transcript_person p ON d.transcriptid = p.transcriptid
JOIN ciq.ciqtranscriptcomponent c ON p.transcriptcomponentid = c.transcriptcomponentid
WHERE d.companyid IN ({ids})
  AND d.mostimportantdateutc >= (CURRENT_DATE - INTERVAL '1 year')
  AND d.keydeveventtypename = 'Earnings Calls'
ORDER BY d.companyid, d.transcriptid, p.componentorder;
"""

In [54]:
raw_segments = db.raw_sql(sql_query)
print("Transcript segments:", len(raw_segments))

Transcript segments: 10255


In [46]:
raw_segments.columns

Index(['companyid', 'transcriptid', 'headline', 'mostimportantdateutc',
       'mostimportanttimeutc', 'keydeveventtypeid', 'keydeveventtypename',
       'detail_companyname', 'transcriptcollectiontypeid',
       'transcriptcomponenttypeid', 'transcriptcomponenttypename',
       'transcriptpersonid', 'transcriptpersonname', 'proid',
       'companyofperson', 'speakertypeid', 'speakertypename', 'componentorder',
       'componenttextpreview', 'componenttext'],
      dtype='object')

In [55]:
raw_segments.loc[:,"timestamp"] = (
    raw_segments["mostimportantdateutc"].astype(str) + " " +
    raw_segments["mostimportanttimeutc"].astype(str)
)

# unique calls by timestamp
unique_call_counts = (
    raw_segments.groupby(["companyid", "detail_companyname"])["timestamp"]
    .nunique()
    .reset_index(name="unique_timestamp_count")
)

unique_call_counts

,companyid,detail_companyname,unique_timestamp_count
0,19691.0,"Cisco Systems, Inc.",3
1,21171.0,Intuit Inc.,4
2,21835.0,Microsoft Corporation,4
3,22247.0,Oracle Corporation,4
4,24321.0,Adobe Inc.,4
5,24937.0,Apple Inc.,4
6,32307.0,NVIDIA Corporation,4
7,33493.0,QUALCOMM Incorporated,4
8,112350.0,International Business Machines Corporation,4
9,122917.0,"Salesforce, Inc.",4


In [58]:
segments_enriched = raw_segments.merge(
    top_tech_with_ciq,
    left_on="companyid",
    right_on="ciq_company_id",
    how="left"
)

call_section = segments_enriched.loc[:,[
    "companyid",
    "company_name",
    "market_cap",
    "transcriptid",
    "headline",
    "mostimportantdateutc",
    "mostimportanttimeutc",
    "keydeveventtypeid",
    "keydeveventtypename",
    "transcriptcollectiontypeid",
    "transcriptcomponenttypename",
    "transcriptpersonname",
    "speakertypename",
    "componentorder",
    "componenttext",
]]

call_section.loc[:,"segment_id"] = (call_section["transcriptid"].astype(str) + "_" + call_section["componentorder"].astype(str))
call_section = call_section.reset_index(drop=True)
call_section["segment_idx"] = call_section.index

In [61]:
call_section["mostimportantdateutc"] = pd.to_datetime(call_section["mostimportantdateutc"], errors="coerce").astype("datetime64[ns]")
call_section.loc[:,"call_year"] = call_section["mostimportantdateutc"].dt.year
call_section.loc[:,"call_quarter"] = "Q" + call_section["mostimportantdateutc"].dt.quarter.astype(str)

call_section.loc[:,"call_period"] = (call_section["call_year"].astype(str) + " " + call_section["call_quarter"])

In [ ]:
import re

def clean_component_text(text: str) -> str:
    if not isinstance(text, str): return ""
    t = text.replace("\r", " ").strip()
    t = re.sub(r"\s+", " ", t)
    return t

call_section.loc[:,"clean_text"] = call_section["componenttext"].apply(clean_component_text)

In [64]:
import os
import pandas as pd

CORPUS_PATH = "data/corpus"
os.makedirs(CORPUS_PATH, exist_ok=True)

CALL_SECTION_PARQUET = os.path.join(CORPUS_PATH, "tech_call_sections.parquet")
call_section.to_parquet(CALL_SECTION_PARQUET, index=False)
print("Saved:", CALL_SECTION_PARQUET)

Saved: data/corpus/tech_call_sections.parquet


In [65]:
call_section["call_datetime"] = pd.to_datetime(
    call_section["mostimportantdateutc"].astype(str) + " " + call_section["mostimportanttimeutc"].astype(str),
    errors="coerce",
)

In [69]:
call_level = (
    call_section
    .sort_values(["companyid", "transcriptid", "componentorder"])
    .groupby(["companyid", "company_name", "market_cap", "transcriptid", "headline", "call_datetime"], as_index=False)
    .agg({"componenttext": lambda parts: "\n".join(parts)})
)

In [70]:
call_level = call_level.rename(columns={"componenttext": "full_transcript"})
call_level["call_date"] = (pd.to_datetime(call_level["call_datetime"], errors="coerce").astype("datetime64[ns]"))
call_level["call_year"] = call_level["call_date"].dt.year
call_level["call_quarter"] = "Q" + call_level["call_date"].dt.quarter.astype(str)
call_level["call_period"] = (call_level["call_year"].astype(str) + " " + call_level["call_quarter"])

EARNINGS_CALL_PARQUET = os.path.join(CORPUS_PATH, "tech_earnings_calls.parquet")
call_level.to_parquet(EARNINGS_CALL_PARQUET, index=False)
print("Saved:", EARNINGS_CALL_PARQUET)

Saved: data/corpus/tech_earnings_calls.parquet


/var/folders/m6/86n33n2x3b5_jv4lcwk88kb80000gn/T/ipykernel_23923/2394348975.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  call_level["call_date"] = (pd.to_datetime(call_level["call_datetime"], errors="coerce").astype("datetime64[ns]"))

In [74]:
call_section.columns

Index(['companyid', 'company_name', 'market_cap', 'transcriptid', 'headline',
       'mostimportantdateutc', 'mostimportanttimeutc', 'keydeveventtypeid',
       'keydeveventtypename', 'transcriptcollectiontypeid',
       'transcriptcomponenttypename', 'transcriptpersonname',
       'speakertypename', 'componentorder', 'componenttext', 'segment_id',
       'segment_idx', 'call_year', 'call_quarter', 'call_period', 'clean_text',
       'call_datetime'],
      dtype='object')

In [75]:
call_section

,companyid,company_name,market_cap,transcriptid,headline,mostimportantdateutc,mostimportanttimeutc,keydeveventtypeid,keydeveventtypename,transcriptcollectiontypeid,...,speakertypename,componentorder,componenttext,segment_id,segment_idx,call_year,call_quarter,call_period,clean_text,call_datetime
0,19691.0,CISCO SYSTEMS INC,269596.8,3372444.0,"Cisco Systems, Inc., Q2 2025 Earnings Call, Fe...",2025-02-12,21:30:00,48.0,Earnings Calls,7,...,Operator,0,[Audio Gap],3372444.0_0,0,2025,Q1,2025 Q1,[Audio Gap],2025-02-12 21:30:00
1,19691.0,CISCO SYSTEMS INC,269596.8,3372444.0,"Cisco Systems, Inc., Q2 2025 Earnings Call, Fe...",2025-02-12,21:30:00,48.0,Earnings Calls,7,...,Executives,1,[Audio Gap] Demand growth in our Nexus portfol...,3372444.0_1,1,2025,Q1,2025 Q1,[Audio Gap] Demand growth in our Nexus portfol...,2025-02-12 21:30:00
2,19691.0,CISCO SYSTEMS INC,269596.8,3372444.0,"Cisco Systems, Inc., Q2 2025 Earnings Call, Fe...",2025-02-12,21:30:00,48.0,Earnings Calls,7,...,Executives,2,"Thank you, And I just want to remind analysts ...",3372444.0_2,2,2025,Q1,2025 Q1,"Thank you, And I just want to remind analysts ...",2025-02-12 21:30:00
3,19691.0,CISCO SYSTEMS INC,269596.8,3372444.0,"Cisco Systems, Inc., Q2 2025 Earnings Call, Fe...",2025-02-12,21:30:00,48.0,Earnings Calls,7,...,Operator,3,"Ben Reitzes, Melius Research.",3372444.0_3,3,2025,Q1,2025 Q1,"Ben Reitzes, Melius Research.",2025-02-12 21:30:00
4,19691.0,CISCO SYSTEMS INC,269596.8,3372444.0,"Cisco Systems, Inc., Q2 2025 Earnings Call, Fe...",2025-02-12,21:30:00,48.0,Earnings Calls,7,...,Analysts,4,Congrats on the quarter. It's good to see you ...,3372444.0_4,4,2025,Q1,2025 Q1,Congrats on the quarter. It's good to see you ...,2025-02-12 21:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10250,43580005.0,PALANTIR TECHNOLOG INC,176883.0659,3576741.0,"Palantir Technologies Inc., Q3 2025 Earnings C...",2025-11-03,22:00:00,48.0,Earnings Calls,2,...,Executives,17,Ryan is our best at being a wonderful -- I'll ...,3576741.0_17,10250,2025,Q4,2025 Q4,Ryan is our best at being a wonderful -- I'll ...,2025-11-03 22:00:00
10251,43580005.0,PALANTIR TECHNOLOG INC,176883.0659,3576741.0,"Palantir Technologies Inc., Q3 2025 Earnings C...",2025-11-03,22:00:00,48.0,Earnings Calls,2,...,Executives,18,I'll make the mistake of trying to follow Alex...,3576741.0_18,10251,2025,Q4,2025 Q4,I'll make the mistake of trying to follow Alex...,2025-11-03 22:00:00
10252,43580005.0,PALANTIR TECHNOLOG INC,176883.0659,3576741.0,"Palantir Technologies Inc., Q3 2025 Earnings C...",2025-11-03,22:00:00,48.0,Earnings Calls,2,...,Executives,19,"Thank you. Alex, as always, we have a lot of i...",3576741.0_19,10252,2025,Q4,2025 Q4,"Thank you. Alex, as always, we have a lot of i...",2025-11-03 22:00:00
10253,43580005.0,PALANTIR TECHNOLOG INC,176883.0659,3576741.0,"Palantir Technologies Inc., Q3 2025 Earnings C...",2025-11-03,22:00:00,48.0,Earnings Calls,2,...,Executives,20,We're rocking along. Please turn on the conven...,3576741.0_20,10253,2025,Q4,2025 Q4,We're rocking along. Please turn on the conven...,2025-11-03 22:00:00
